# Limpeza e Tratamento de dados do dataset

## 1) Imports

In [13]:
# Imports de bibliotecas
import pandas as pd
import numpy as np
import missingno as mso
import seaborn as sns
import matplotlib as plt
import scipy as sp
import kagglehub
import os

# Import dos dados brutos do Kaggle
path = kagglehub.dataset_download("mos3santos/conjunto-de-dados-de-preos-de-carros")
df = pd.read_csv(path + "/car_price_dataset.csv", low_memory=False)
df.head()

,Brand,Model,Year,Engine_Size,Fuel_Type,Transmission,Mileage,Doors,Owner_Count,Price
0,Kia,Rio,2020,4.2,Diesel,Manual,289944,3,5,8501
1,Chevrolet,Malibu,2012,2.0,Hybrid,Automatic,5356,2,3,12092
2,Mercedes,GLA,2020,4.2,Diesel,Automatic,231440,4,2,11171
3,Audi,Q5,2023,2.0,Electric,Manual,160971,2,1,11780
4,Volkswagen,Golf,2003,2.6,Hybrid,Semi-Automatic,286618,3,3,2867


### Salvando dados brutos na pasta raw

In [21]:
# Cópia do dataset no estado bruto
raw = df.copy()

diretorio = '../dados/raw'
nome_arquivo = 'car_price_dataset_raw.csv'
caminho_completo = os.path.join(diretorio, nome_arquivo)

# Criar diretório se não existir
os.makedirs(diretorio, exist_ok=True)

# Verificar se já existe
if os.path.isfile(caminho_completo):
    print(f"Arquivo {nome_arquivo} já existia, será sobrescrito.")
else:
    print(f"Criando novo arquivo: {nome_arquivo}")

# Salvar
raw.to_csv(caminho_completo, index=False)
print(f"Arquivo salvo em: {caminho_completo}")

Arquivo car_price_dataset_raw.csv já existia, será sobrescrito.
Arquivo salvo em: ../dados/raw\car_price_dataset_raw.csv


## 2) Análise de Missing Values

In [22]:
# Verificando condições dos dados
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Brand         10000 non-null  object 
 1   Model         10000 non-null  object 
 2   Year          10000 non-null  int64  
 3   Engine_Size   10000 non-null  float64
 4   Fuel_Type     10000 non-null  object 
 5   Transmission  10000 non-null  object 
 6   Mileage       10000 non-null  int64  
 7   Doors         10000 non-null  int64  
 8   Owner_Count   10000 non-null  int64  
 9   Price         10000 non-null  int64  
dtypes: float64(1), int64(5), object(4)
memory usage: 781.4+ KB


In [17]:
# Verificando nulos
df.isnull().sum()

Brand           0
Model           0
Year            0
Engine_Size     0
Fuel_Type       0
Transmission    0
Mileage         0
Doors           0
Owner_Count     0
Price           0
dtype: int64

> Aparententemente, o dataset não necessita de tratamento de valores nulos, uma vez que o mesmo não possui nenhum em uma coluna sequer com nulos evidentes

> Cabe a investigação para saber se não há valores correspondentes aos dados nulos, como 0s em colunas numéricas ou strings representativas em colunas categóricas

### Verificação de valores representativos a nulos

In [18]:
# Verificando se há valores representativos a nulos em colunas categóricas
for coluna in df.columns:
    if (df[coluna].dtype) == object:
        print(df[coluna].unique())

['Kia' 'Chevrolet' 'Mercedes' 'Audi' 'Volkswagen' 'Toyota' 'Honda' 'BMW'
 'Hyundai' 'Ford']
['Rio' 'Malibu' 'GLA' 'Q5' 'Golf' 'Camry' 'Civic' 'Sportage' 'RAV4'
 '5 Series' 'CR-V' 'Elantra' 'Tiguan' 'Equinox' 'Explorer' 'A3' '3 Series'
 'Tucson' 'Passat' 'Impala' 'Corolla' 'Optima' 'Fiesta' 'A4' 'Focus'
 'E-Class' 'Sonata' 'C-Class' 'X5' 'Accord']
['Diesel' 'Hybrid' 'Electric' 'Petrol']
['Manual' 'Automatic' 'Semi-Automatic']


In [19]:
# Verificando em colunas numéricas
for coluna in df.columns:
    if not(df[coluna].dtype) == object:
        print(df[coluna].unique())

[2020 2012 2023 2003 2007 2010 2001 2014 2005 2013 2009 2004 2017 2006
 2000 2021 2018 2016 2011 2022 2008 2015 2002 2019]
[4.2 2.  2.6 2.7 3.4 4.7 3.1 1.3 4.5 4.8 2.3 2.4 3.2 2.9 1.5 1.8 4.6 2.8
 3.9 3.3 1.9 3.5 1.4 2.2 1.  1.2 4.9 3.6 3.8 4.  1.7 1.6 4.1 1.1 2.5 3.
 5.  2.1 3.7 4.3 4.4]
[289944   5356 231440 ...  86664 225772 157882]
[3 2 4 5]
[5 3 2 1 4]
[ 8501 12092 11171 ...  9866  4084  3342]


In [23]:
# Verificação em colunas numéricas de inputação 0 / -1 no lugar de nulos
for coluna in df.columns:
    if not(df[coluna].dtype) == object:
        if (df[coluna] == 0).any() or (df[coluna] == -1).any():
            print(f"A coluna {coluna} possui valores 0 ou -1, que podem representar nulos.")
        else:
            print(f"A coluna {coluna} não possui valores 0 ou -1.")

A coluna Year não possui valores 0 ou -1.
A coluna Engine_Size não possui valores 0 ou -1.
A coluna Mileage não possui valores 0 ou -1.
A coluna Doors não possui valores 0 ou -1.
A coluna Owner_Count não possui valores 0 ou -1.
A coluna Price não possui valores 0 ou -1.


> Não há indícios de substituição de valores nulos por algum valor representativo

> A base de dados já estava anteriormente limpa em relação a valores nulos

## 3) Análise de Duplicatas

In [24]:
df.duplicated().sum()

np.int64(0)

> O dataset também não possui dados duplicados

## 4) Análise de Dados Incongruentes

In [25]:
df.describe()

,Year,Engine_Size,Mileage,Doors,Owner_Count,Price
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000
mean,2011.543700,3.000560,149239.111800,3.497100,2.991100,8852.96440
std,6.897699,1.149324,86322.348957,1.110097,1.422682,3112.59681
min,2000.000000,1.000000,25.000000,2.000000,1.000000,2000.00000
25%,2006.000000,2.000000,74649.250000,3.000000,2.000000,6646.00000
50%,2012.000000,3.000000,149587.000000,3.000000,3.000000,8858.50000
75%,2017.000000,4.000000,223577.500000,4.000000,4.000000,11086.50000
max,2023.000000,5.000000,299947.000000,5.000000,5.000000,18301.00000


> Não há inconsistências nos dados, portanto a estruturação da limpeza pode ser encerrada sem nenhuma ação tomada, pois os dados já estavam limpos

## 5) Salvando dados "limpos" para EDA:

In [32]:
# Cópia do dataset no estado "limpo"
clean = df.copy()

diretorio = '../dados/processed'
nome_arquivo = 'car_price_dataset_clean.csv'
caminho_completo = os.path.join(diretorio, nome_arquivo)

# Criar diretório se não existir
os.makedirs(diretorio, exist_ok=True)

# Verificar se já existe
if os.path.isfile(caminho_completo):
    print(f"Arquivo {nome_arquivo} já existia, será sobrescrito.")
else:
    print(f"Criando novo arquivo: {nome_arquivo}")

# Salvar
clean.to_csv(caminho_completo, index=False)
print(f"Arquivo salvo em: {caminho_completo}")

Arquivo car_price_dataset_clean.csv já existia, será sobrescrito.
Arquivo salvo em: ../dados/processed\car_price_dataset_clean.csv
